In [1]:
using Test, pulse_input_DDM, LinearAlgebra, Flatten, Parameters

In [2]:
n, cross = 53, false

(53, false)

In [3]:
## Choice model
θ = θchoice(θz=θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 50., σ2_s = 1.5,
    ϕ = 0.8, τ_ϕ = 0.05),
    bias=1., lapse=0.05)

θ, data = synthetic_data(;θ=θ, ntrials=10, rng=1)
model_gen = choiceDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6))

choiceDDM{θchoice{θz{Float64},Float64},Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}},1},θprior{Float64}}
  θ: θchoice{θz{Float64},Float64}
  data: Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false
  θprior: θprior{Float64}


In [4]:
choices = getfield.(data, :choice);

In [5]:
@test all(choices .== vcat(falses(9), trues(1)))

Test Passed

In [6]:
@time @test round(loglikelihood(model_gen), digits=2) ≈ -3.72

  1.910217 seconds (4.66 M allocations: 251.721 MiB, 9.61% gc time)


Test Passed

In [7]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 14.32

Test Passed

In [8]:
options = choiceoptions(lb=vcat([0., 8.,  -5., 0.,   0.,  0.01, 0.005], [-30, 0.]),
    ub = vcat([2., 30., 5., 100., 2.5, 1.2,  1.], [30, 1.]), 
    fit = trues(dimz+2));

In [9]:
model, = optimize(data, options; iterations=5, outer_iterations=1, θprior=θprior(μ_B=40., σ_B=1e6));
@test round(norm(Flatten.flatten(model.θ)), digits=2) ≈ 25.05

Fminbox
-------
Initial mu = 0.000239451

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000239451

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     2.212393e+01     9.828961e+01
 * time: 4.792213439941406e-5

Exiting inner optimizer with x = [0.12345655322543442, 14.99895530690319, -0.13726257383746795, 19.99805424621317, 0.6951082089308465, 1.1588961674091682, 0.0063088058501385305, 0.6814442212420025, 0.500394967367097]
Current distance to box: 0.00130881
Decreasing barrier term μ.



Test Passed

In [10]:
H = Hessian(model)
@test round(norm(H), digits=2) ≈ 92.6

Test Passed

In [11]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 1471.54

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 8.46979164813496
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed

In [12]:
## Neural model
ncells, ntrials = [1,2], [10,5]
f = [repeat(["Sigmoid"], N) for N in ncells]
                    
θ = θneural(θz = θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 10., σ2_s = 1.2,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid() for n in 1:N] for N in ncells], f=f);

In [13]:
data, = synthetic_data(θ, ntrials, ncells);
model_gen = neuralDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6));

In [14]:
spikes = map(x-> sum.(x), getfield.(vcat(data...), :spikes))

15-element Array{Array{Int64,1},1}:
 [3]     
 [14]    
 [6]     
 [1]     
 [5]     
 [9]     
 [20]    
 [9]     
 [5]     
 [21]    
 [11, 10]
 [5, 8]  
 [7, 5]  
 [10, 11]
 [11, 9] 

In [15]:
@test all(spikes .== [[3], [14], [6], [1], [5], [9], [20], [9], [5], [21], [11, 10], [5, 8], [7, 5], [10, 11], [11, 9]])

Test Passed

In [16]:
@test round(loglikelihood(model_gen), digits=2) ≈ -529.86

Test Passed

In [17]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 52.18

Test Passed

In [18]:
x = pulse_input_DDM.flatten(θ)
@test round(loglikelihood(x, model_gen), digits=2) ≈ -529.86

Test Passed

In [19]:
θy0 = vcat(vcat(θy.(data, f)...)...)
@test round(norm(θy0), digits=2) ≈ 31.16

Test Passed

In [20]:
#deterministic model
options0 = neural_options_noiseless(f)

neural_options_noiseless
  fit: Array{Bool}((19,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((19,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((19,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [21]:
x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0)

19-element Array{Float64,1}:
   0.0                  
  30.0                  
   2.220446049250313e-16
   0.0                  
   0.0                  
   0.9999999999999998   
   0.008                
  -7.705663789255116    
   7.678951595360267    
   0.0011352713910126341
   0.0                  
 -11.526171720551112    
  11.655646461267072    
   0.008735402516937699 
   0.0                  
  -8.170615489116662    
  22.738475419974247    
   0.0075096756861132   
   0.0                  

In [22]:
θ0 = θneural_noiseless(x0, f)
model0 = noiseless_neuralDDM(θ0, data)

noiseless_neuralDDM{θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))


In [23]:
@test round(loglikelihood(model0), digits=2) ≈ -1399.68

Test Passed

In [24]:
x0 = pulse_input_DDM.flatten(θ0)
@unpack f = θ0

θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Sigmoid{Float64},1}}((2,))
  f: Array{Array{String,1}}((2,))


In [25]:
@test round(loglikelihood(x0, model0), digits=2) ≈ -1399.68

Test Passed

In [26]:
model, = optimize(model0, options0; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 87.51

Test Passed

In [27]:
@test round(norm(gradient(model)), digits=2) ≈ 6.82

Test Passed

In [28]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])

19-element Array{Float64,1}:
  0.1               
 15.0               
 -0.1               
 20.0               
  0.5               
  0.8               
  0.008             
 52.69070630256158  
 43.434427853058395 
 -2.8625905261748006
  5.0336085210607395
 17.047935166694103 
 33.8056973114395   
  5.1937297903733795
  3.97561896305667  
 -5.690347702664896 
 23.203791176447094 
  0.4792342779693217
  3.6141666197326594

In [29]:
options = neural_options(f)  

model = neuralDDM(θneural(x0, f), data, n, cross, θprior(μ_B=40., σ_B=1e6))
model, = optimize(model, options; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 85.73

Fminbox
-------
Initial mu = 0.00248599

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00248599

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.827493e+02     7.356733e+02
 * time: 5.91278076171875e-5

Exiting inner optimizer with x = [1.051828368475773, 15.03668934024119, 0.2797507123339492, 20.04006641872528, 0.5926364853980968, 0.01838739503368525, 0.7082387560405169, 52.42451557332407, 43.40460710623912, -3.3663760919374215, 4.683811149066144, 16.995759054681873, 33.792099189665144, 4.917694876411467, 3.9073304947867564, -5.689492847672851, 23.20298505742793, 0.280912565861838, 3.6334905003779103]
Current distance to box: 0.0173874
Decreasing barrier term μ.



Test Passed

In [30]:
H = Hessian(model; chunk_size=4)
@test round(norm(H), digits=2) ≈ 22.83

Test Passed

In [31]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 303.03

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 1.7824957110747937
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed

In [32]:
options = neural_choice_options(f)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [57]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false


In [58]:
@test round(choice_loglikelihood(choice_neural_model), digits=2) ≈ -36.54

Test Passed

In [59]:
@test round(joint_loglikelihood(choice_neural_model), digits=2) ≈ -784.56

Test Passed

In [60]:
import pulse_input_DDM: nθparams
nparams, = nθparams(f)

([4, 4, 4], [1, 2])

In [61]:
fit = vcat(falses(dimz), trues(2), falses.(nparams)...);
options = neural_choice_options(fit=fit, lb=options.lb, ub=options.ub)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  ub: Array{Float64}((21,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [62]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [2]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/UkDyx/src/multivariate/solvers/constrained/fminbox.jl:227


Fminbox
-------
Initial mu = 0.00197924

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00197924

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     2.553943e+01     1.985415e+02
 * time: 3.600120544433594e-5

Exiting inner optimizer with x = [-0.028033421445001122, 0.8068812577669018]
Current distance to box: 0.193119
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-2.80e-02, 8.07e-01]
    Minimum:   1.121763e+01

 * Found with
    Algorithm:     Fminbox with BFGS
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    |x - x'|               = 7.97e-01 ≰ 1.0e-10
    |x - x'|/|x'|          = 9.88e-01 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.43e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.28e+00 ≰ 1.0e-09
    |g(x)|                 = 5.04e+00 ≰ 1.0e-03

 * Work counters
    Seconds run:   1  (vs limit 170000)
    Iterations:    1
    f(x) calls:    11
    ∇f(x) calls:   3
)

In [63]:
@test round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits=2) ≈ 85.93

Test Passed

In [64]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false


In [65]:
fit = vcat(trues(dimz), trues(2), trues.(nparams)...);
options = neural_choice_options(fit=fit, lb=vcat(options.lb[1:7], -10., options.lb[9:end]), 
    ub=vcat(options.ub[1:7], 10., options.ub[9:end]))

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [66]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [9]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/UkDyx/src/multivariate/solvers/constrained/fminbox.jl:227


Fminbox
-------
Initial mu = 0.00110367

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00110367

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     2.543751e+01     2.650124e+02
 * time: 0.00010204315185546875

Exiting inner optimizer with x = [0.09999524442638871, 14.999998861579506, -0.1000353495211857, 19.999998148622932, 0.49989346551840624, 0.8001036938544792, 0.005421400935056857, 1.7818291128839726e-5, 0.011742648809349407, 52.690705268133655, 43.43442319573108, -2.8625997880185126, 5.033602894087483, 17.047933839120894, 33.80569633912378, 5.1937304084987606, 3.9756182221025242, -5.690348609616332, 23.203790628041897, 0.4792654757364251, 3.6141619017081927]
Current distance to box: 0.000421401
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [1.00e-01, 1.50e+01, -1.00e-01,  ...]
    Minimum:   2.451307e+01

 * Found with
    Algorithm:     Fminbox with BFGS
    Initial Point: [1.00e-01, 1.50e+01, -1.00e-01,  ...]

 * Convergence measures
    |x - x'|               = 3.12e-03 ≰ 1.0e-10
    |x - x'|/|x'|          = 3.63e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.02e+00 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.20e-02 ≰ 1.0e-09
    |g(x)|                 = 2.78e+02 ≰ 1.0e-03

 * Work counters
    Seconds run:   4  (vs limit 170000)
    Iterations:    1
    f(x) calls:    43
    ∇f(x) calls:   3
)

In [67]:
@test round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits=2) ≈ 85.93

Test Failed at In[67]:1
  Expression: round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits=2) ≈ 85.93
   Evaluated: 85.92 ≈ 85.93


Test.FallbackTestSetException: There was an error during testing